In [ ]:
import matplotlib.pyplot as plt
import torch
import torchvision


from torch import nn

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchinfo import summary
import modules.data_setup as data_setup
from modules.engine import train_step,test_step,train
from modules.create_writer import create_writer
from modules.predict_plot import device, pred_and_plot_image
from modules.save_model import save_model
import os


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

In [ ]:
version = torch.__version__
print(f"PyTorch Version: {version}")

In [ ]:
import requests
from pathlib import Path
import zipfile

data_path = Path('data')
data_20_percent_path = data_path / 'pizza_steak_sushi_20_percent'

if data_20_percent_path.is_dir():
    print(f'{data_20_percent_path} exists')
else:
    print(f'{data_20_percent_path} does not exist, creating...')
    data_20_percent_path.mkdir(parents=True, exist_ok=True)

    with open(data_path / 'pizza_steak_sushi_20_percent.zip','wb') as f:
         request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip")
         print('Downloading pizza_steak_sushi_20_percent data')
         f.write(request.content)
    with zipfile.ZipFile(data_path / 'pizza_steak_sushi_20_percent.zip','r') as zip_ref:
        print("Extracting pizza_steak_sushi_20_percent.zip")
        zip_ref.extractall(data_20_percent_path)

    os.remove(data_path / 'pizza_steak_sushi_20_percent.zip')

In [ ]:
def set_seed(seed:int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [ ]:
train_dir = data_20_percent_path/'train'
test_dir = data_20_percent_path/'test'
normalize = transforms.Normalize(
    mean=[0.485,0.456,0.406],
    std=[0.229,0.224,0.225]
)
simple_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])
print(simple_transform)


In [ ]:
train_dataloader,test_dataloader,class_names = data_setup.create_dataloaders(
                                                train_dir=train_dir,
                                                test_dir=test_dir,
                                                train_transform=simple_transform,
                                                test_transform=simple_transform,
                                                batch_size=32)
train_dataloader,test_dataloader,class_names

In [ ]:
weights =  torchvision.models.EfficientNet_B1_Weights.DEFAULT
effnetb1 = torchvision.models.efficientnet_b1(weights=weights)
model_summary = summary(
    model=effnetb1,
    input_size=(32,3,224,224),
    verbose=0,
    col_names = ['input_size', 'output_size', 'num_params','trainable'],
    col_width=20,
    row_settings=['var_names']
)
model_summary

In [ ]:
for param in effnetb1.features.parameters():
    param.requires_grad=False

set_seed()
effnetb1.classifier = nn.Sequential(
    nn.Dropout(p=0.2,inplace=True),
    nn.Linear(in_features=1280,out_features=len(class_names),bias=True)
)

In [ ]:
model_summary = summary(
    model=effnetb1,
    input_size=(32,3,224,224),
    verbose=0,
    col_names = ['input_size', 'output_size', 'num_params','trainable'],
    col_width=20,
    row_settings=['var_names']
)
model_summary

In [ ]:
experiment_number = 0
set_seed()
model_name = "effnetb1"
num_epochs = [5,10]
for epochs in num_epochs:
    experiment_number += 1
    print(f"[INFO] Experiment number: {experiment_number}")
    print(f"[INFO] Model: {model_name}")
    print(f"[INFO] Number of epochs: {epochs}")

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=effnetb1.parameters(),lr=0.001)

    train(
    model = effnetb1,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=epochs,
    device=device,
    writer=create_writer(
        experiment_name="data_20_percent",
        model_name=model_name,
        extra=f"{epochs}_epochs"
    ))

    save_path = f'{model_name}_data_20_percent_{epochs}_epochs.pth'
    save_model(
    model=effnetb1,
    target_dir="models",
    model_name=save_path
    )
    print("-"*50 + "\n")


From the experiment info, we find out efficient-b1 at epoch 10 has the lowest test loss and highest test accuracy. so effnetb1_data_20_percent_10_epochs.pth is the best model here

Load the best model and make predictions with it

In [ ]:
best_model_path = "models/effnetb1_data_20_percent_10_epochs.pth"

best_model = effnetb1

best_model.load_state_dict(torch.load(best_model_path))

In [ ]:
#test model by using a random list of 5 images from test set
from modules.predict_plot import pred_and_plot_image
import random
import matplotlib.pyplot as plt
from typing import List, Tuple

num_images_to_plot = 5
test_image_path_list = list(Path(data_20_percent_path/"test").glob("*/*.jpg"))
test_image_path_sample = random.sample(population=test_image_path_list,k=num_images_to_plot)
for image_path in test_image_path_sample:
    pred_and_plot_image(
        model=best_model,
        image_path=image_path,
        device=device,
        image_size=(224,224),
        class_names=class_names,
    )